In [39]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pprint
pp = pprint.PrettyPrinter(indent=1)

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
num_classes = 5
input_size = 5 # one-hot size 
hidden_size = 5 #output from the cell. 바로 결과 예측을 위해서 5로 설정 
batch_size = 1 # sentence의 개수 (단어개수) 
sequence_length = 1 # 이번에는 한번에 하나씩 해본다 
num_layers = 1 # one layer rnn이다. (아직 안 다룬 개념)

cuda


In [41]:
idx2char = ['h','i','e','l','o']
x_data = [0,1,0,2,3,3] #hihell 

one_hot_lookup = [[1,0,0,0,0], #h 
                  [0,1,0,0,0], 
                  [0,0,1,0,0], 
                  [0,0,0,1,0], 
                  [0,0,0,0,1]] 

x_one_hot = [one_hot_lookup[x] for x in x_data] 
pp.pprint(x_one_hot)

[[1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 1, 0]]


In [42]:
y_data = [1,0,2,3,3,4] #ihello 
inputs = torch.Tensor(x_one_hot) 
labels = torch.LongTensor(y_data)

In [43]:
class Model(nn.Module): 
    def __init__(self): 
        super(Model,self).__init__() 
        self.rnn=nn.RNN(input_size = input_size, \
                        hidden_size=hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x, hidden): 
        #input x 를 (batch_size,sequence_length,input_size)로 reshape
        x = x.view(batch_size, sequence_length, input_size) 
        
        #Propagate input through RNN 
        #Input:(batch,seq_len,input_size) 
        out,hidden = self.rnn(x, hidden) 
        
        #for make sure, output이 N * 5 shape을 따르게 하기 위해서 
        #out = out.view(-1, num_classes)
        
        return hidden, out 
    
    def init_hidden(self): 
        #initialize hidden and cell states 
        
        return torch.zeros(num_layers,batch_size,hidden_size).to(device)

In [44]:
model = Model().to(device) 

criterion = nn.CrossEntropyLoss() #1 
optimizer = optim.Adam(model.parameters(),lr = 0.1) 

In [38]:
for epoch in range(100): 

    optimizer.zero_grad() 
    loss = 0 
    hidden = model.init_hidden() #2 
    print("predicted string : ",end="") 
    
    for input_,label in zip(inputs, labels): #3
        inputs = inputs.to(device)
        labels = labels.to(device)
        labels = labels.view(-1,1)
        hidden, output = model(input_, hidden)
        output = output.view(-1, num_classes)
        val, idx = output.max(1) 
        
        print(idx2char[idx.data[0]],end="") 
        loss += criterion(output,label) 
    
    print(", epoch: %d, loss: %1.3f" % (epoch+1, loss.item())) 
    loss.backward() 
    optimizer.step() 



predicted string : lllooo, epoch: 1, loss: 10.045
predicted string : lllloo, epoch: 2, loss: 9.045
predicted string : lllllo, epoch: 3, loss: 8.293
predicted string : lllllo, epoch: 4, loss: 7.629
predicted string : llello, epoch: 5, loss: 6.979
predicted string : lhello, epoch: 6, loss: 6.452
predicted string : ihello, epoch: 7, loss: 6.066
predicted string : ihello, epoch: 8, loss: 5.727
predicted string : ihello, epoch: 9, loss: 5.428
predicted string : ihello, epoch: 10, loss: 5.184
predicted string : ihello, epoch: 11, loss: 4.967
predicted string : ihello, epoch: 12, loss: 4.767
predicted string : ihello, epoch: 13, loss: 4.608
predicted string : ihello, epoch: 14, loss: 4.495
predicted string : ihello, epoch: 15, loss: 4.410
predicted string : ihello, epoch: 16, loss: 4.324
predicted string : ihello, epoch: 17, loss: 4.240
predicted string : ihello, epoch: 18, loss: 4.164
predicted string : ihello, epoch: 19, loss: 4.095
predicted string : ihello, epoch: 20, loss: 4.030
predicte

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
num_classes = 5
input_size = 5 # one-hot size 
hidden_size = 5 #output from the cell. 바로 결과 예측을 위해서 5로 설정 
batch_size = 1 # sentence의 개수 (단어개수) 
sequence_length = 6 # 이번에는 한번에 6개씩 해본다 
num_layers = 1 # one layer rnn이다. (아직 안 다룬 개념)

idx2char = ['h','i','e','l','o']
x_data = [0,1,0,2,3,3] #hihell 

one_hot_lookup = [[1,0,0,0,0], #h 
                  [0,1,0,0,0], 
                  [0,0,1,0,0], 
                  [0,0,0,1,0], 
                  [0,0,0,0,1]] 

x_one_hot = [one_hot_lookup[x] for x in x_data] 
print(x_one_hot)

cuda
[[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 1, 0]]


```python
    def forward(self, x, hidden): 
        #input x 를 (batch_size,sequence_length,input_size)로 reshape
        x = x.view(batch_size, sequence_length, input_size) 
        
        #Propagate input through RNN 
        #Input:(batch,seq_len,input_size) 
        out,hidden = self.rnn(x, hidden) 
        
        #for make sure, output이 N * 5 shape을 따르게 하기 위해서 
        out = out.view(-1, num_classes) 
        
        return hidden, out 
    ```

```python
for epoch in range(100): 

    optimizer.zero_grad() 
    loss = 0 
    hidden = model.init_hidden() #2 
    print("predicted string : ",end="") 
    
    for input_,label in zip(inputs, labels): #3
        inputs = inputs.to(device)
        labels = labels.to(device)
        labels = labels.view(-1,1)
        hidden, output = model(input_, hidden)

        val, idx = output.max(1) 
        
        print(idx2char[idx.data[0]],end="") 
        loss += criterion(output,label) 
    
    print(", epoch: %d, loss: %1.3f" % (epoch+1, loss.item())) 
    loss.backward() 
    optimizer.step() 

```

In [46]:
model = Model().to(device) 

criterion = nn.CrossEntropyLoss() #1 
optimizer = optim.Adam(model.parameters(),lr = 0.1) 

In [49]:
for epoch in range(100): 
    optimizer.zero_grad() 
    loss = 0 
    hidden = model.init_hidden() 
    
    # print("predicted string : ",end="") 
    inputs = inputs.to(device) 
    labels = labels.to(device) 
    
    _, outputs = model(inputs,hidden) 
    
    outputs = outputs.view(-1, 5)
    loss = criterion(outputs, labels.squeeze()) 
    _, idx = outputs.max(1) 

    result_str = [idx2char[c] for c in idx.squeeze()] 
    print("epoch: %d, loss: %1.3f" % (epoch+1,loss.item())) 
    print("Predicted string: ", ''.join(result_str)) 
    loss.backward() 
    optimizer.step()



epoch: 1, loss: 1.553
Predicted string:  ieelii
epoch: 2, loss: 1.390
Predicted string:  ilelle
epoch: 3, loss: 1.281
Predicted string:  ilelll
epoch: 4, loss: 1.199
Predicted string:  ililll
epoch: 5, loss: 1.112
Predicted string:  ililll
epoch: 6, loss: 1.039
Predicted string:  ihilll
epoch: 7, loss: 0.974
Predicted string:  ihioll
epoch: 8, loss: 0.915
Predicted string:  ihioll
epoch: 9, loss: 0.866
Predicted string:  ihioll
epoch: 10, loss: 0.824
Predicted string:  iheoll
epoch: 11, loss: 0.781
Predicted string:  iheolo
epoch: 12, loss: 0.735
Predicted string:  iheolo
epoch: 13, loss: 0.685
Predicted string:  ihello
epoch: 14, loss: 0.642
Predicted string:  ihello
epoch: 15, loss: 0.614
Predicted string:  ihello
epoch: 16, loss: 0.586
Predicted string:  ihello
epoch: 17, loss: 0.568
Predicted string:  ihello
epoch: 18, loss: 0.561
Predicted string:  ihello
epoch: 19, loss: 0.554
Predicted string:  ihello
epoch: 20, loss: 0.544
Predicted string:  ihello
epoch: 21, loss: 0.535
Predic